# 1. Environment Setup

In [ ]:
import numpy as np
import pandas as pd

# Resources will be available on local/ directory

import os
import requests, zipfile, io, os, shutil, subprocess
github_repo = 'alejoUdeAMl/ml_har_v1'
zip_file_url = 'https://github.com/alejoUdeAMl/ml_har_v1/archive/main.zip'
if not os.path.exists("local"):
    print("replicating local resources")
    dirname = github_repo.split("/")[-1] + "-main/"
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    if os.path.exists("local"):
        shutil.rmtree("local")
    if os.path.exists(dirname + "/content/local"):
        shutil.move(dirname + "/content/local", "local")
    elif os.path.exists(dirname + "/local"):
        shutil.move(dirname + "/local", "local")
    shutil.rmtree(dirname)

# 2. Human Action Recognition (HAR)
* Human activity recognition, or HAR for short, is a broad field of study concerned with identifying the specific movement or action of a person based on sensor data.
* Movements are often typical activities performed indoors, such as walking, talking, standing, and sitting


# Why it is important ?
* Human activity recognition plays a significant role in human-to-human interaction and interpersonal relations.
* Because it provides information about the identity of a person, their personality, and psychological state, it is difficult to extract.
* The human ability to recognize another person’s activities is one of the main subjects of study of the scientific areas of computer vision and machine learning. As a result of this research, many applications, including video surveillance systems, human-computer interaction, and robotics for human behavior characterization, require a multiple activity recognition system.
* On my research I will need to clasify patients actions on a monitored environment.

# 3. Importing libraries

In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from PIL import Image
import matplotlib.image as img
import matplotlib.pyplot as plt

# 4. Getting the path and Loading the data

In [ ]:
train_data = pd.read_csv("local/Human Action Recognition/Training_set.csv")
test_data = pd.read_csv("local/Human Action Recognition/Testing_set.csv")

In [ ]:
train_fol = glob.glob("local/Human Action Recognition/train/*") 
test_fol = glob.glob("local/Human Action Recognition/test/*")

In [ ]:
train_data

In [ ]:
train_data.label.value_counts()

In [ ]:
import plotly.express as px
HAR = train_data.label.value_counts()
fig = px.pie(train_data, values=HAR.values, names=HAR.index, title='Pie of Human Actions')
fig.show()

In [ ]:
filename = train_data['filename']

situation = train_data['label']

In [ ]:
filename

In [ ]:
situation

# 5. Display random image

In [ ]:
def displaying_random_images():
    num = random.randint(1, 10000)
    imgg = "Image_{}.jpg".format(num)
    train = "local/Human Action Recognition/train/"
    if os.path.exists(train + imgg):
        testImage = img.imread(train + imgg)
        plt.imshow(testImage)
        plt.title("{}".format(train_data.loc[train_data['filename'] == "{}".format(imgg), 'label'].item()))

    else:
        print("File Path not found \nSkipping the file!!")

In [ ]:
displaying_random_images()

# 6. Resizing images

In [ ]:
img_data = []
img_label = []
length = len(train_fol)
for i in (range(len(train_fol)-1)):
    t = 'local/Human Action Recognition/train/' + filename[i]    
    temp_img = Image.open(t)
    img_data.append(np.asarray(temp_img.resize((240,240))))
    img_label.append(situation[i])

In [ ]:
img_shape= (240,240,3)

In [ ]:
iii = img_data
iii = np.asarray(iii)
type(iii)

In [ ]:
y_train = to_categorical(np.asarray(train_data["label"].factorize()[0]))
print(y_train[0])

# 7. Define CNN model with EfficientNetB7

In [ ]:
efficientnet_model = Sequential()

base_model = tf.keras.applications.EfficientNetB7(include_top=False,
                                            input_shape=(240,240,3),
                                            pooling ="avg",classes=15,
                                             weights="imagenet")

for layer in base_model.layers:
    layer.trainable=False
    

efficientnet_model.add(base_model)
efficientnet_model.add(Flatten())
efficientnet_model.add(Dense(512, activation="relu"))
efficientnet_model.add(Dense(15, activation="softmax"))

In [ ]:
efficientnet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
efficientnet_model.summary()

# 8. Train the model

In [ ]:
history = efficientnet_model.fit(iii,y_train, epochs=20)

In [ ]:
losses = history.history["loss"]
plt.plot(losses)

In [ ]:
acc = history.history['accuracy']
plt.plot(acc)

# 9. Model predictions

In [ ]:
def read_img(fn):
    img = Image.open(fn)
    return np.asarray(img.resize((160,160)))

In [ ]:
classes_dict = train_data.label.value_counts().index.tolist()

In [ ]:
def test_predict(test_image):
    result = efficientnet_model.predict(np.asarray([read_img(test_image)]))

    itemindex = np.where(result == np.max(result))
    prediction = itemindex[1][0]
    print("probability: " + str(np.max(result)*100) + "%\nPredicted class : ", classes_dict[prediction])

    image = img.imread(test_image)
    plt.imshow(image)
    plt.title(prediction)

In [ ]:
test_predict("local/Human Action Recognition/test/Image_1001.jpg")

In [ ]:
test_predict("local/Human Action Recognition/test/Image_101.jpg")

In [ ]:
test_predict("local/Human Action Recognition/test/Image_1056.jpg")

In [ ]:
test_predict("local/Human Action Recognition/test/Image_1024.jpg")

In [ ]:
def predict_random_image():
    num = random.randint(1,5400)
    imgg = "Image_{}.jpg".format(num)
    train = "local/Human Action Recognition/test/"
    path = train + imgg
    if os.path.exists(path):
        test_predict(path)
    else:
        print("File Path not found \nSkipping the file!!")

In [ ]:
predict_random_image()